In [1]:
import numpy as np

In [39]:
# 2.1: solve system of linear equations
from scipy.optimize import linprog

# linear inequalities: A * x <= b
def get_entire_point(A, b):
    n = len(A[0])
    bounds_for_coordinates = [(None, None) for i in range(n)]
    coefficients_x_i = [0 for i in range(n)]  
    
    def get_x_i_bounds(i):
        bounds = None
        coefficients_x_i[i] = 1
        res_min = linprog(coefficients_x_i,
                          A_ub=A,
                          b_ub=b,
                          bounds=bounds_for_coordinates)
        coefficients_x_i[i] = -1
        res_max = linprog(coefficients_x_i,
                          A_ub=A,
                          b_ub=b,
                          bounds=bounds_for_coordinates)
        coefficients_x_i[i] = 0
        min_x_i = res_min.fun
        max_x_i = -res_max.fun
        if res_min.status == 3 and res_max.status == 3:
            bounds = [-1, 1]
        elif res_min.status == 3:
            bounds = [max_x_i - 2, max_x_i]
        elif res_max.status == 3:
            bounds = [min_x_i, min_x_i + 2]
        elif res_min.status == 0 and res_max.status == 0:
            bounds = [min_x_i, max_x_i]
        return bounds
    
    entire_point = []
    for i in range(n):
        bounds = get_x_i_bounds(i)
        if bounds is None or bounds[0] == bounds[1]:
            return None
        x_i = (bounds[0] + bounds[1]) / 2
        entire_point.append(x_i)
        bounds_for_coordinates[i] = (x_i, x_i)
    return entire_point

#test
entire_pt = get_entire_point([[1, 1], [-1, -1]], [3, -2])
assert entire_pt == [0, 2.5] 
print(entire_pt)

entire_pt = get_entire_point([[-1, 0], [0, -1], [1, 1]], [0, 0, 1])
assert entire_pt == [0.5, 0.25] 
print(entire_pt)

entire_pt = get_entire_point([[1, 0], [-1, 0]], [0, 0])
assert entire_pt is None
print(entire_pt)

entire_pt = get_entire_point([[1, 0], [-1, 0]], [-2, 0])
assert entire_pt is None
print(entire_pt)

entire_pt = get_entire_point([[1, 0], [-1, 0]], [2, 0])
assert entire_pt == [1, 0]
print(entire_pt)

[0.0, 2.5]
[0.5, 0.25]
None
None
[1.0, 0.0]
